#A/B Test Analysis: Website Conversion Optimization

# Stage 1: Data Environment Setup and Initial Ingestion

### Description
The first phase of the project involves setting up the analytical environment in Google Colab and importing the dataset. This stage ensures that the Python environment has access to the data stored on Google Drive and validates that the file structure is correct for subsequent statistical analysis.

### Key Technical Steps
* **Environment Connection:** Mounting Google Drive and importing the `pandas` and `os` libraries to manage files and dataframes.
* **Data Loading:** Using `pd.read_csv()` to load the A/B test results into a DataFrame named `df`.
* **Structural Inspection:** Executing `df.head()` to preview the first rows and `df.info()` to check for data types, non-null counts, and memory usage.

### Findings
* **Dataset Confirmation:** The file `ab_test_results.csv` was successfully loaded.
* **Data Integrity:** The initial inspection shows that the dataset contains key columns: `ga_session_id` (unique session identifier), `test_group` (numerical group assignment), and `event_name` (user interactions).
* **Readiness:** No immediate data type conversions are required, and the dataframe is ready for cleaning and validation.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.listdir('/content/drive/MyDrive/')

In [ ]:
import pandas as pd

file_name = 'ab_test_results (1).csv'
df = pd.read_csv(file_name)
print("The file was imported successfully.")
display(df.head())
print("\nColumn metadata:")
print(df.info())

In [ ]:
df = pd.read_csv('ab_test_results (1).csv')
print("Data snippet:")
display(df.head())
print("\nInformacje o typach danych:")
print(df.info())

# Stage 2: Statistical Significance Analysis (Global KPIs)

### Description
In this phase, we evaluate the global performance of the A/B test. The goal is to determine if the observed differences in user behavior between the Control and Test groups are statistically significant.

### Technical Methodology
* **Mapping:** Numerical group IDs (`1` and `2`) are mapped to descriptive labels (`control` and `test`) for better clarity.
* **Denominator:** We calculate the total number of unique sessions per group to establish a baseline.
* **Proportion Z-Test:** We apply a two-proportion Z-test to compare conversion rates. The test helps us distinguish between a genuine performance lift and random statistical noise.
* **KPIs:** The analysis focuses on four key metrics: `add_payment_info`, `add_shipping_info`, `begin_checkout`, and `new accounts`.

In [ ]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

group_map = {1: 'control', 2: 'test'}
df['group_name'] = df['test_group'].map(group_map)
sessions_per_group = df[df['event_name'] == 'session'].groupby('group_name')['ga_session_id'].nunique()
metrics = [
    'add_payment_info',
    'add_shipping_info',
    'begin_checkout',
    'new accounts'
]

def run_stat_test(df, event_name, sessions):
    conversions = df[df['event_name'] == event_name].groupby('group_name')['ga_session_id'].nunique()

    if 'test' not in conversions or 'control' not in conversions:
        return {'Metric': event_name, 'Status': 'Missing Data'}

    counts = np.array([conversions['test'], conversions['control']])
    totals = np.array([sessions['test'], sessions['control']])

    stat, pval = proportions_ztest(counts, totals)

    return {
        'Metric': f"{event_name} / session",
        'Control CR': f"{(counts[1]/totals[1]):.2%}",
        'Test CR': f"{(counts[0]/totals[0]):.2%}",
        'P-Value': f"{pval:.4f}",
        'Is Significant?': 'YES' if pval < 0.05 else 'No'
    }

final_results = [run_stat_test(df, m, sessions_per_group) for m in metrics]
english_summary_df = pd.DataFrame(final_results)

print("A/B Test Statistical Analysis Results:")
display(english_summary_df)

### Analysis & Conclusions (Global Results)
* **Statistical Significance:** None of the analyzed metrics reached the significance threshold of **p < 0.05**. This means we cannot confidently state that the changes in the Test group had any impact on global conversion rates.
* **Funnel Performance:** The conversion rates for checkout-related events (payment, shipping, and initiation) remained nearly identical between both groups, with a marginal difference of only ~0.02%.
* **Account Creation:** The `new accounts` metric showed a slight decrease in the Test group (from 8.43% to 8.29%), but with a P-Value of 0.117, this result is still considered statistically insignificant.
* **Final Verdict:** At a global level, the A/B test is **neutral**. There is no evidence of a "winner," which justifies a deeper look into specific segments (countries or devices).

# Stage 3: Granular Segment Analysis (Dimensions Breakdown)

### Description
In this final analytical stage, we move beyond global averages to perform a deep dive into the data by breaking down the results into specific dimensions: `country`, `device`, `channel`, and `continent`. This approach is essential because an experiment that appears neutral at a global scale may have hidden winning or losing segments that are critical for business decisions.

### Technical Methodology
* **Dynamic Segmentation:** Instead of hard-coding specific values, the code automatically identifies all unique segments within the columns: `country`, `device`, `channel`, and `continent`.
* **Automated Looping:** We utilize a Python loop to iterate through every unique segment, creating a filtered subset of data for each iteration.
* **Reusable Function:** The `calculate_significance` function is called repeatedly for each segment, ensuring consistent calculation of the Z-test and Conversion Rates.
* **Safety Logic:** The function includes a check to skip segments that do not have enough data (e.g., sessions missing in either the Control or Test group) to prevent mathematical errors and ensure reliability.

In [ ]:
target_metrics = ['add_payment_info', 'add_shipping_info', 'begin_checkout', 'new accounts']
dimensions = ['country', 'device', 'channel', 'continent']

def calculate_significance(data, metric_list):
    """
    Calculates A/B test results for a given slice of data.
    """
    # Calculate total sessions (denominator) per group
    # Group 1 = Control, Group 2 = Test
    sessions = data[data['event_name'] == 'session'].groupby('test_group')['ga_session_id'].nunique()

    # Ensure both groups exist in the current data slice
    if 1 not in sessions or 2 not in sessions:
        return []

    results = []
    for metric in metric_list:
        # Count unique sessions with the event (numerator)
        conversions = data[data['event_name'] == metric].groupby('test_group')['ga_session_id'].nunique()

        count_control = conversions.get(1, 0)
        count_test = conversions.get(2, 0)
        total_control = sessions[1]
        total_test = sessions[2]

        # Statistical Z-test
        counts = np.array([count_test, count_control])
        totals = np.array([total_test, total_control])

        # Avoid errors if zero conversions
        if count_control == 0 or count_test == 0:
            stat, pval = (0, 1.0)
        else:
            stat, pval = proportions_ztest(counts, totals)

        results.append({
            'Metric': f"{metric} / session",
            'Control CR': count_control / total_control,
            'Test CR': count_test / total_test,
            'P-Value': pval,
            'Significant': 'YES' if pval < 0.05 else 'No'
        })
    return results

# --- EXECUTION ---

# A. Global Significance (Whole Test)
print("--- CALCULATING GLOBAL SIGNIFICANCE ---")
global_results = calculate_significance(df, target_metrics)
global_summary = pd.DataFrame(global_results)
display(global_summary)

# B. Breakdown Significance (Loops for countries, devices, etc.)
print("\n--- CALCULATING BREAKDOWNS (By Country, Device, Channel) ---")
all_breakdown_results = []

for dim in dimensions:
    unique_values = df[dim].dropna().unique()
    for val in unique_values:
        # Filter data for the specific segment (e.g., Country == 'Poland')
        subset = df[df[dim] == val]

        # Run significance for this segment
        segment_results = calculate_significance(subset, target_metrics)

        # Add metadata and store
        for res in segment_results:
            res['Dimension'] = dim
            res['Segment'] = val
            all_breakdown_results.append(res)

breakdown_summary = pd.DataFrame(all_breakdown_results)

breakdown_summary = breakdown_summary[['Dimension', 'Segment', 'Metric', 'Control CR', 'Test CR', 'P-Value', 'Significant']]
display(breakdown_summary.head(20)) # Displaying first 20 results

### Analysis & Conclusions (Segmented Results)

* **Consistency Across Segments:** The segmented analysis largely mirrors the global results. In most key markets such as **Serbia**, **Georgia**, and **Israel**, the changes between groups were negligible, with P-Values remaining high.
* **Interesting Observations - Romania:** Romania stands out as an interesting case. For `add_payment_info`, the Test group conversion rate (1.47%) is nearly triple that of the Control group (0.54%). However, with a P-Value of **0.12**, this does not yet meet the rigorous standards for statistical significance (p < 0.05).
* **Zero-Impact Segments:** In certain segments, like **Georgia**, the conversion rates remained identical across both groups (P-Value: 1.0), indicating that the test variation had absolutely no effect on user behavior in that region.
* **Final Summary of Python Phase:** * Globally: **No Significant Winner.**
    * Segment-wise: **No Significant Winner identified.**
    * **Next Step:** Despite the lack of statistical significance, we will now proceed to visualize these distributions in **Tableau**. Visualization often reveals trends and "near-significant" patterns that are difficult to spot in raw tables, which can be valuable for future hypothesis generation.

# Stage 4: Data Export for Visualization

### Description
The final step in our Python workflow is to export the processed statistical results. By saving the `breakdown_summary` table as a CSV file, we ensure that all complex calculations (Conversion Rates, P-Values, and Significance status) are preserved and ready for professional visualization in Tableau.

### Technical Steps
* **CSV Serialization:** Using the `.to_csv()` method to convert the DataFrame into a portable file format.
* **File Download:** Utilizing the `google.colab.files` module to trigger a local download of the results directly to the user's computer.

In [ ]:
breakdown_summary.to_csv('ab_test_final_results.csv', index=False)
from google.colab import files
files.download('ab_test_final_results.csv')

### Summary of the Python Phase
* **Analytical Readiness:** We have successfully transitioned from raw log data to a structured, statistically-verified summary table.
* **Data Portability:** The exported file `ab_test_final_results.csv` contains all the necessary dimensions and metrics for interactive dashboarding.
* **Final Note:** While the Python analysis confirmed a lack of global significance, the exported dataset is rich enough to allow for deep-dive explorations and visual storytelling in the next phase of the project.

# Visualization and Interactive Reporting in Tableau

### Project Description and Methodology
In this final phase, the statistical results derived from the Python analysis were transformed into an interactive "Business Cockpit" dashboard. This allows stakeholders to explore the A/B test performance across various dimensions and metrics dynamically.

* **Data Source:** The `ab_test_final_results.csv` file generated during the Python analysis phase.
* **Statistical Foundation:** Z-Test for Proportions with a 95% Confidence Level ($\alpha = 0.05$).
* **Tools Used:** Google Colab (Python/Statsmodels) and Tableau Desktop/Public.

### Project Links
* **Interactive Tableau Dashboard:** [https://public.tableau.com/app/profile/jan.cwirko/viz/ABTESTANALYSISPORTFOLIO/Dashboard1?publish=yes]
* **Final Results Dataset (CSV):** [https://drive.google.com/file/d/1PzOPhP-SNpZifnjUQv060AJO_N8yoUF_/view?usp=sharing]

### Project Development Stages
* **Dashboard Architecture:** Designed a unique sidebar-driven layout with a navy-blue control panel to separate user inputs from the data display.
* **Interactivity & Navigation:** Implemented "Single Value List" filters for both `Metric` and `Dimension`, enabling precise segmentation (e.g., by Acquisition Channel or Continent).
* **Visual Optimization:** Removed grid lines and row banding to create a lightweight, modern interface, focusing attention on key KPIs like `P-Value` and `Test CR`.
* **Data Alignment:** Centered all table contents and expanded the dashboard width to ensure all statistical values are readable without horizontal scrolling.

### Final Insights and Recommendations
* **Lack of Statistical Significance:** Most test variations yielded a `P-Value > 0.05`, indicating that the observed differences between the Control and Test groups are likely due to random chance rather than the implemented changes.
* **Segment Exploration:** While the global results are not significant, the dashboard highlights stable conversion rates in specific segments like `Organic Search` and `Direct`, providing a baseline for future testing.
* **Business Recommendation:** Business Recommendation: Conclude the current test and reject the hypothesis that the variant changes significantly impact conversion rates. It is recommended to iterate on the test design or explore different hypotheses based on the stable segments identified in the dashboard.